# Your first scraper

- Repository: `Wikipedia Scrapper`
- Type of Challenge: `Learning`
- Duration: `3 days`
- Submission: `23/06/2023 4:30 PM`

- Team challenge : `Solo`

## The Mission

In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

## Learning objectives

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

* Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.
* Activate it.
* You will find more info about virtual environments in the course content and on the web.
* Note: BEFORE pushing changes into your repo explore what the .gitignore file is and add your environment to it. 

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [1]:
# Import the requests library (1 line)
import requests
# Assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.onrender.com"
# Assign the /status endpoint to another variable called status_url (1 line)
status_url = "https://country-leaders.onrender.com/status"
# Query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(status_url)

# Print response if successfull
if req.status_code == 200:
    print(req)
# Print status code if failed
else:
    print(req.status_code)

### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted. The nice thing for us is that JSON files follow a format we already know in Python...dictionaries!

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [6]:
# Set the countries_url variable (1 line)
countries_url = "https://country-leaders.onrender.com/countries"
# Query the /countries endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(countries_url)
# Get the JSON content and store it in the countries variable (1 line)
countries = ""
# Display the request's status code and the countries variable (1 line)
print(req.status_code, countries)

403 


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [15]:
# Set the cookie_url variable (1 line)
cookie_url = "https://country-leaders.onrender.com/cookie"
# Query the enpoint, set the cookies variable and display it (2 lines)
cookies = (requests.get(cookie_url)).cookies
print(cookies)


<RequestsCookieJar[<Cookie user_cookie=b12b7867-edd8-44df-8f2a-d3a94d546dc6 for country-leaders.onrender.com/>]>


Try to query the countries endpoint using the cookie, save the output and print it.

In [29]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries = requests.get(countries_url, cookies=cookies)
# display the countries variable (1 line)
print(countries)

<Response [403]>


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [11]:
# Set the leaders_url variable (1 line)
leaders_url = root_url + "/leaders"
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders = requests.get(leaders_url)
# display the leaders variable (1 line)
print(leaders)

<Response [403]>


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [19]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
# assign the output to the leaders variable (1 line)
cookies = (requests.get(cookie_url)).cookies
leaders = requests.get(leaders_url+"?country=be", cookies=cookies)
# display the leaders variable (1 line)
print(leaders)

<Response [200]>


### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [56]:
# 4 lines
leaders_per_country = {}
countries = ((requests.get(countries_url, cookies=((requests.get(cookie_url)).cookies))).text).replace("[","").replace("]","").replace('"',"").split(",")
for country in countries:
    leaders_per_country[country] = (requests.get(leaders_url+"?country="+country, cookies=((requests.get(cookie_url)).cookies))).text

In [8]:
# or 1 line


It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [104]:
# < 15 lines
def get_leaders():
    cookie_url = "https://country-leaders.onrender.com/cookie"
    countries_url = "https://country-leaders.onrender.com/countries"
    leaders_url = "https://country-leaders.onrender.com/leaders"
    leaders_per_country = {}
    cookies=(requests.get(cookie_url)).cookies
    countries = (requests.get(countries_url, cookies=cookies)).json()
    for country in countries:
        url = leaders_url+"?country="+country
        leaders_per_country[country] = (requests.get(url, cookies=cookies)).json()
    return leaders_per_country

https://country-leaders.onrender.com/leaders?country=us
https://country-leaders.onrender.com/leaders?country=ma
https://country-leaders.onrender.com/leaders?country=be
https://country-leaders.onrender.com/leaders?country=ru
https://country-leaders.onrender.com/leaders?country=fr


{'us': [{'id': 'Q23',
   'first_name': 'George',
   'last_name': 'Washington',
   'birth_date': '1732-02-22',
   'death_date': '1799-12-14',
   'place_of_birth': 'Westmoreland County',
   'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington',
   'start_mandate': '1789-04-30',
   'end_mandate': '1797-03-04'},
  {'id': 'Q76',
   'first_name': 'Barack',
   'last_name': 'Obama',
   'birth_date': '1961-08-04',
   'death_date': None,
   'place_of_birth': 'Kapiolani Medical Center for Women and Children',
   'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama',
   'start_mandate': '2009-01-20',
   'end_mandate': '2017-01-20'},
  {'id': 'Q91',
   'first_name': 'Abraham',
   'last_name': 'Lincoln',
   'birth_date': '1809-02-12',
   'death_date': '1865-04-15',
   'place_of_birth': 'Sinking Spring Farm',
   'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln',
   'start_mandate': '1861-03-04',
   'end_mandate': '1865-04-15'},
  {'id': 'Q207',
   'first_name': 'Ge

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [109]:
# 2 lines
leaders_per_country = get_leaders()
print(leaders_per_country)

https://country-leaders.onrender.com/leaders?country=us
https://country-leaders.onrender.com/leaders?country=ma
https://country-leaders.onrender.com/leaders?country=be
https://country-leaders.onrender.com/leaders?country=ru
https://country-leaders.onrender.com/leaders?country=fr
{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', '

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [131]:
# 3 lines
def get_text():
    leader_url = leaders_per_country['us'][2]['wikipedia_url']
    req = requests.get(leader_url)
    return req.text




Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [154]:
# 3 lines
from bs4 import BeautifulSoup as soup
wikipedia = soup(get_text(), 'html.parser')
#print(wikipedia.prettify())

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [134]:
# 2 lines
for p in wikipedia.find_all('p'):
    print(p)

<p class="mw-empty-elt">
</p>
<p><b>Abraham Lincoln</b> (<span class="rt-commentedText nowrap"><span class="IPA nopopups noexcerpt" lang="en-fonipa"><a href="/wiki/Help:IPA/English" title="Help:IPA/English">/<span style="border-bottom:1px dotted"><span title="/ˈ/: primary stress follows">ˈ</span><span title="'l' in 'lie'">l</span><span title="/ɪ/: 'i' in 'kit'">ɪ</span><span title="/ŋ/: 'ng' in 'sing'">ŋ</span><span title="'k' in 'kind'">k</span><span title="/ən/: 'on' in 'button'">ən</span></span>/</a></span></span> <a href="/wiki/Help:Pronunciation_respelling_key" title="Help:Pronunciation respelling key"><i title="English pronunciation respelling"><span style="font-size:90%">LINK</span>-ən</i></a>; February 12, 1809 – April 15, 1865) was an American lawyer, politician, and statesman who served as the 16th <a href="/wiki/President_of_the_United_States" title="President of the United States">president of the United States</a> from 1861 until <a href="/wiki/Assassination_of_Abraham_Lin

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [212]:
# <10 lines
from bs4 import BeautifulSoup as soup
def get_text():
    leader_url = leaders_per_country['us'][2]['wikipedia_url']
    req = requests.get(leader_url)
    print(leader_url)
    return req.text

wikipedia = soup(get_text(), 'html.parser')

import re
title =  ""
for p in wikipedia.find_all('p'):
    if re.match("(^<p><b>(.*?)<\/b>)", str(title)):
        first_paragraph = p
        print(first_paragraph)
        break
    title = p




https://en.wikipedia.org/wiki/Abraham_Lincoln
<p>Lincoln was born into poverty in a <a href="/wiki/Log_cabin" title="Log cabin">log cabin</a> in <a href="/wiki/Kentucky" title="Kentucky">Kentucky</a> and was raised on the <a href="/wiki/American_frontier" title="American frontier">frontier</a>, primarily in <a href="/wiki/Indiana" title="Indiana">Indiana</a>. He was self-educated and became a lawyer, <a href="/wiki/Whig_Party_(United_States)" title="Whig Party (United States)">Whig Party</a> leader, <a href="/wiki/Illinois" title="Illinois">Illinois</a> state <a href="/wiki/Illinois_House_of_Representatives" title="Illinois House of Representatives">legislator</a>, and <a class="mw-redirect" href="/wiki/U.S._Congressman" title="U.S. Congressman">U.S. Congressman</a> <a href="/wiki/List_of_United_States_representatives_from_Illinois" title="List of United States representatives from Illinois">from Illinois</a>. In 1849, he returned to his successful law practice in <a href="/wiki/Spring

At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [228]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    #print(wikipedia_url) # keep this for the rest of the notebook
    from bs4 import BeautifulSoup as soup
    url = requests.get(wikipedia_url).text
    wikipedia = soup(url, 'html.parser')
    import re

    for p in wikipedia.find_all('p'):
        if re.match("(^<p><b>(.*?)<\/b>)", str(p)):
            first_paragraph = re.sub("<.*?>", "", str(p))
            return first_paragraph
            break

In [252]:
# Test: 3 lines
i = 0
for rows in leaders_per_country['us']:
    result = get_first_paragraph(leaders_per_country['us'][i]['wikipedia_url'])
    i+=1
    result = re.sub("\(\/.*?\;", "(", str(result))
    result = re.sub("\[.\]", "", str(result))
    print(result)

George Washington (February 22, 1732 – December 14, 1799) was an American military officer, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797. Appointed by the Continental Congress as commander of the Continental Army, Washington led Patriot forces to victory in the American Revolutionary War and served as president of the Constitutional Convention of 1787, which created and ratified the Constitution of the United States and the American federal government. Washington has been called the "Father of his Country" for his manifold leadership in the nation's founding.[10]

Barack Hussein Obama II ( born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American  president of the United States. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 199

KeyboardInterrupt: 

### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [233]:
# 3 lines
result = re.sub("<.*?>", "", str(result))
print(result)


Harry S. Truman[b] (May 8, 1884 – December 26, 1972) was the 33rd president of the United States, serving from 1945 to 1953. A leader of the Democratic Party, he previously served as the 34th vice president from January to April 1945 under Franklin D. Roosevelt and as a United States senator from Missouri from 1935 to January 1945. Assuming the presidency after Roosevelt's death, Truman implemented the Marshall Plan to rebuild the economy of Western Europe and established both the Truman Doctrine and NATO to contain the expansion of Soviet communism. He proposed numerous liberal domestic reforms, but few were enacted by the conservative coalition that dominated the Congress.



Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [18]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    #print(wikipedia_url) # keep this for the rest of the notebook
    from bs4 import BeautifulSoup as soup
    url = requests.get(wikipedia_url).text
    wikipedia = soup(url, 'html.parser')
    import re

    for p in wikipedia.find_all('p'):
        if re.match("(^<p><b>(.*?)<\/b>)", str(p)):
            first_paragraph = re.sub("<.*?>", "", str(p))
            return first_paragraph
            break

Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [261]:
# < 20 lines
def get_first_paragraph(wikipedia_url):
    #print(wikipedia_url) # keep this for the rest of the notebook
    from bs4 import BeautifulSoup as soup
    url = requests.get(wikipedia_url).text
    wikipedia = soup(url, 'html.parser')
    import re

    for p in wikipedia.find_all('p'):
        if re.match("(^<p><b>(.*?)<\/b>)", str(p)):
            # remove tags
            first_paragraph = re.sub("<.*?>", "", str(p))
            # remove phonetics
            first_paragraph = re.sub("\(\/.*?\;", "(", str(first_paragraph))
            # remove [n]
            first_paragraph = re.sub("\[.\]", "", str(first_paragraph))
            # remove trailing \n
            first_paragraph = re.sub("\\n", "", str(first_paragraph))
            return first_paragraph
            break
i = 0
for rows in leaders_per_country['us']:
    result = get_first_paragraph(leaders_per_country['us'][i]['wikipedia_url'])
    i+=1
    print(result)

George Washington (February 22, 1732 – December 14, 1799) was an American military officer, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797. Appointed by the Continental Congress as commander of the Continental Army, Washington led Patriot forces to victory in the American Revolutionary War and served as president of the Constitutional Convention of 1787, which created and ratified the Constitution of the United States and the American federal government. Washington has been called the "Father of his Country" for his manifold leadership in the nation's founding.[10]
Barack Hussein Obama II ( born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American  president of the United States. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997

KeyboardInterrupt: 

## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [20]:
# < 20 lines

In [21]:
# Check the output of your function (2 lines)


Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [22]:
# < 25 lines



Check the output of your function again.

In [23]:
# Check the output of your function (1 line)


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [24]:
# < 20 lines


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [25]:
# <25 lines


Test your new functions.



## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [26]:
# 3 lines


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [27]:
# 3 lines


Make a function `save(leaders_per_country)` to call this code easily.

In [28]:
# 3 lines


In [29]:
# Call the function (1 line)


## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

### Constraints

#### Code style

- Your code should be **commented**. You can use a docstring to explain what your functions are doing. E.g. 

```python
def add(number_one, number_two):
    """
    Function that will perform the add operation between two numbers (in params).
    """
    result = number_one + number_two
    return result
```


- Your code should be **cleaned of any commented, unused code**.

### GitHub Repository

- Push your code on a GitHub repository. The name is at the top of this page.
- Your repository should only contain the files specified.
- Your `README.md` should be nice and complete.
  It should include the following outline:
  - Description (What is the project about? What is the goal? What is your approach?)
  - Installation (What are the requirements to use your code?)
  - Usage (How does someone use your code?)
  - Timeline (How long did the project take?)
  - Personal situation (e.g. This project was done as part of the AI Boocamp at BeCode.org.)

Feel free to add more information.

## Evaluation criteria

| Criteria       | Indicator                                                   | Yes/No |
| -------------- | ----------------------------------------------------------- | ------ |
| 1. Is complete | The student has realized all must-have features.            |        |
|                | There is a published GitHub repo available.                 |        |
|                | The scrapper starts by running `python main.py` in the terminal |        |
| 2. Is correct  | The code is well typed.                                     |        |
|                | All the constraints are respected                           |        |


## To go further (if time permits)

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!

![wikepedia_gif](https://media.giphy.com/media/mADzY6J0QjniQl9gs5/giphy.gif)